# Our RSA Encryption

## Math Functions for RSA

In [1]:
def gcd(a, b):
    assert type(a) == int, "a not an integer"
    assert type(b) == int, "b not an integer"
    assert a >= 0, "a is not positve"
    assert b >= 0, "b is not positive"
    
    # terminating case: if second number is zero,
    # we found gcd
    if b == 0:
        return a
    return gcd(b, a % b)

def pulverizer(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = pulverizer(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = pulverizer(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m
    
def is_prime(p):
    for i in range(2, (p//2)+1):
        if (p % i) == 0:
            return False
    else:
        return True
    
def is_relatively_prime(p, q):
    return gcd(p, q) == 1

## RSA Functions

In [2]:
phrases = {
    1: "Have a nice day!",
    2: "The weather has been disappointing.",
    3: "Is cereal a soup?",
    4: "Is a hotdog a sandwich?",
    5: "Yes", 
    6: "No",
    7: "I'm not sure.",
    8: "I want to learn more programming.",
    9: "I love The Lion King!",
    10: "I think Boss Baby is better."
}

In [3]:
def create_keys(p, q, e):
    # Step 1
    assert is_prime(p), "p is not prime!"
    assert is_prime(q), "q is not prime!"
    
    # Step 2
    n = p * q
    z = (p-1) * (q-1)
    
    # Step 3
    assert is_relatively_prime(e, z)
    
    # Step 4
    d = modinv(e, z)
    
    public_key = (e, n)
    private_key = (d, n)
    
    return public_key, private_key
    
def encrypt(msg, public_key):
    e, n = public_key
    msg_prime = (msg**e) % n
    return msg_prime

def decrypt(msg_prime, private_key):
    d, n = private_key
    msg = (msg_prime**d) % n
    return phrases[msg]

In [4]:
public_key, private_key = create_keys(17, 11, 13)

In [5]:
print(public_key)

(13, 187)


In [6]:
encrypted_message = encrypt(4, public_key)

In [7]:
decrypt(encrypted_message, private_key)

'Is a hotdog a sandwich?'

## RSA Classes

In [8]:
class Key:
    def __init__(self, p, q, e):
        assert is_prime(p), "p is not prime!"
        assert is_prime(q), "q is not prime!"
        self.n = p * q
        
    def get_n(self):
        return self.n
    
    def __repr__(self):
        return f"Key(n = {self.n})"
    
    __str__ = __repr__
    
class PublicKey(Key):
    def __init__(self, n, e):
        self.n = n
        self.e = e
        
    def get_e(self):
        return self.e
    
    def __repr__(self):
        return f"PublicKey(n = {self.n}, e = {self.e})"
    
    __str__ = __repr__
    
    
class PrivateKey(Key):
    def __init__(self, n, d):
        self.n = n
        self.d = d
    
    def __repr__(self):
        return f"PrivateKey(n = {self.n}, d = ********)"
    
    __str__ = __repr__   

In [9]:
class RSA:
    def __init__(self, p, q, e, phrases):
        assert is_prime(p), "p is not prime!"
        assert is_prime(q), "q is not prime!"
        
        z = (p-1) * (q-1)
        assert is_relatively_prime(e, z), "e is invalid!"
        
        d = modinv(e, z)
        self.n = p * q
        self.e = e
        self.d = d
        self.public_key = None
        self.private_key = None
        self.phrases = phrases
        
    def generate_keys(self):
        self.public_key = PublicKey(self.n, self.e)
        self.private_key = PrivateKey(self.n, self.d)
        return (
                PrivateKey(self.n, self.d),
                PublicKey(self.n, self.e)
               )
    
    def encrypt(self, m):
        e = self.public_key.get_e()
        n = self.public_key.get_n()
        m_prime = (m**e) % n
        return m_prime

    def decrypt(self, m_prime):
        n = self.private_key.get_n()
        m = (m_prime**self.d) % n
        return self.phrases[m]
    
    def __repr__(self):
        return f"RSA(n = {self.n}, e = {self.e})"
    
    __str__ = __repr__

In [10]:
rsa_env = RSA(13, 19, 11, phrases)
priv, pub = rsa_env.generate_keys()
print(priv)
print(pub)

PrivateKey(n = 247, d = ********)
PublicKey(n = 247, e = 11)


In [11]:
hidden_message = rsa_env.encrypt(7)

In [12]:
true_message = rsa_env.decrypt(hidden_message)

In [13]:
true_message

"I'm not sure."